# Run simple simulation
## Load module

In [1]:
import PyRAMSES

## Load simulator

In [2]:
ram = PyRAMSES.sim()

## Load saved test-case

In [3]:
case = PyRAMSES.cfg('cmd.txt')

## Add one observation more

In [4]:
case.addRunObs('MS g2') # will plot in real-time the voltage on bus g1

## Run simulation and pause at t=15 seconds

In [ ]:
start_time=15
ram.execSim(case,start_time)
comp_type = ['TOR']
comp_name = ['g14']
prm_name = ['Tm0']
ram.getPrm(comp_type, comp_name, prm_name)

1.000002941834732

## Initialization

In [ ]:
comp_type = ['SYN']
comp_name = ['g2']
obs_name = ['Omega']

kp = 0.25
ki = 0.15
errSum = 0
t=240
nominal_frequency = 1.0
list_of_gens = ['g14', 'g15', 'g16', 'g17', 'g18']

## PI Control

In [ ]:
for i in range(start_time+1,t):
    #print("i = " + str(i))
    actual_frequency = ram.getObs(comp_type,comp_name, obs_name)[0]
    error = nominal_frequency - actual_frequency
    if abs(error)<0.00001:
        error = 0.0
    #print("error = " + str(error))
    errSum += error * 1.0
    #print("errSum = " + str(errSum))
    output = kp * error + ki * errSum
    if abs(output)<0.00001:
        output = 0.0
    # loop to send measurements to generators g14-g18
    for gen in list_of_gens:
        command = 'CHGPRM TOR ' + gen + ' Tm0 ' + str(output/5.0) + ' 0'
        print(str(ram.getSimTime()+0.01)+' '+command)
        ram.addDisturb(ram.getSimTime()+0.01, command)
    ram.contSim(i)

15.010999999999786 CHGPRM TOR g14 Tm0 0.0002790538026191935 0
15.010999999999786 CHGPRM TOR g15 Tm0 0.0002790538026191935 0
15.010999999999786 CHGPRM TOR g16 Tm0 0.0002790538026191935 0
15.010999999999786 CHGPRM TOR g17 Tm0 0.0002790538026191935 0
15.010999999999786 CHGPRM TOR g18 Tm0 0.0002790538026191935 0
16.01 CHGPRM TOR g14 Tm0 0.00037811999412394766 0
16.01 CHGPRM TOR g15 Tm0 0.00037811999412394766 0
16.01 CHGPRM TOR g16 Tm0 0.00037811999412394766 0
16.01 CHGPRM TOR g17 Tm0 0.00037811999412394766 0
16.01 CHGPRM TOR g18 Tm0 0.00037811999412394766 0
17.01 CHGPRM TOR g14 Tm0 0.00047522879180898254 0
17.01 CHGPRM TOR g15 Tm0 0.00047522879180898254 0
17.01 CHGPRM TOR g16 Tm0 0.00047522879180898254 0
17.01 CHGPRM TOR g17 Tm0 0.00047522879180898254 0
17.01 CHGPRM TOR g18 Tm0 0.00047522879180898254 0
18.01 CHGPRM TOR g14 Tm0 0.0005694157403939359 0
18.01 CHGPRM TOR g15 Tm0 0.0005694157403939359 0
18.01 CHGPRM TOR g16 Tm0 0.0005694157403939359 0
18.01 CHGPRM TOR g17 Tm0 0.0005694157403939

In [ ]:
ram.contSim(ram.getInfTime())